In [25]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [26]:
data = pd.read_csv("post_EDA.csv")
data.shape

(230188, 16)

In [27]:
null = data.isnull().sum()
null_sorted = null.sort_values(ascending = False)
print(null_sorted)
data = data.dropna()

severity          0
speed_limit       0
intersection      0
weather           0
lighting          0
month             0
day_of_week       0
hour              0
pedestrian        0
vehicle_other     0
car               0
large_vehicles    0
rail_vehicle      0
two_wheeled       0
time_of_day       0
season            0
dtype: int64


In [28]:
X  = data.drop(columns = "severity")
Y = data['severity'] - 1

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 123, test_size = .3)

In [30]:
log_model = LogisticRegression()

In [31]:
scaler = StandardScaler()

In [32]:
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [33]:
X_test_scaled = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

In [34]:
X_train_final, y_train_final = SMOTE().fit_resample(X_train_scaled, y_train)
np.bincount(y_train_final)

array([102910, 102910, 102910], dtype=int64)

# **Log Reg**

In [35]:
steps =  [('pca', PCA(n_components=3)), ('m', LogisticRegression())]
model = Pipeline(steps = steps)

In [36]:
model.fit(X_train_scaled, y_train)

Pipeline(steps=[('pca', PCA(n_components=3)), ('m', LogisticRegression())])

In [37]:
y_pred_log = model.predict(X_test_scaled)

In [38]:
print(classification_report(y_test, y_pred_log))

              precision    recall  f1-score   support

           0       0.67      0.93      0.78     44262
           1       0.51      0.15      0.24     23302
           2       0.00      0.00      0.00      1493

    accuracy                           0.65     69057
   macro avg       0.39      0.36      0.34     69057
weighted avg       0.60      0.65      0.58     69057



C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
def create_model(neurons= 5, optimizer = 'Adam', activation = 'sigmoid', layer_count = 2, learning_rate = .003):
  model = Sequential()    
  model.add(Dense(neurons, activation = activation, input_shape = (15, ), kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5)))
  for i in range(layer_count - 1):    
    model.add(Dense(neurons, activation=activation, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5)))
  model.add(Dense(1, activation='hard_sigmoid'))  
  model.compile(loss='categorical_crossentropy', optimizer= optimizer, metrics=['categorical_accuracy'])
  K.set_value(model.optimizer.learning_rate, learning_rate)
  return model

In [40]:
#nn = create_model()
#nn.fit(X_train_scaled, y_train, epochs = 200, verbose = 0)

In [41]:
#y_pred_net = nn.predict(X_test_scaled)

In [42]:
#y_pred_net = np.argmax(y_pred_net, axis=-1)

In [43]:
#print(classification_report(y_test, y_pred_net))

In [44]:
from sklearn.utils import resample

params = {
    'criterion' : ['ginni', 'entropy'],
    'n_estimators' : [10, 50, 100, 200],
    'max_depth' : [4, 5, 6, 10, 15, 20],
    'max_features' : [None, 'sqrt', 'log2'],
    'min_impurity_decrease' : [0, 0.0005, 0.0007, 0.001, 0.01],
    'bootstrap' : [False, True]
}
search = GridSearchCV(RandomForestClassifier(), params, n_jobs=-2, cv=5)
X_train_sampled, y_train_sampled = resample(X_train_scaled, y_train)
model = search.fit(X_train_sampled, y_train_sampled)
best = model.best_estimator_.get_params()

C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
3600 fits failed out of a total of 7200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1800 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\ensemble\_forest.py", line 476, in fit
    trees = Parallel(
  File "C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):

In [45]:
rf = RandomForestClassifier(n_estimators = best['n_estimators'], 
                               max_depth = best['max_depth'],
                               max_features = best['max_features'], 
                               min_impurity_decrease = best['min_impurity_decrease'],
                               bootstrap = best['bootstrap'],
                               criterion = best['criterion']
                              )
rf.fit(X_train_sampled, y_train_sampled)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=20,
                       min_impurity_decrease=0)

In [46]:
y_pred_rf = rf.predict(X_test_scaled)

In [47]:
#y_pred_rf = np.argmax(y_pred_rf, axis=-1)

In [48]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.73      0.87      0.79     44262
           1       0.61      0.42      0.50     23302
           2       0.54      0.20      0.29      1493

    accuracy                           0.70     69057
   macro avg       0.63      0.50      0.53     69057
weighted avg       0.69      0.70      0.68     69057



In [49]:
dt = DecisionTreeClassifier()

In [50]:
dt.fit(X_train_scaled, y_train)

DecisionTreeClassifier()

In [51]:
y_pred_dt = dt.predict(X_test_scaled)

In [52]:
print(classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

           0       0.76      0.84      0.80     44262
           1       0.62      0.49      0.55     23302
           2       0.39      0.31      0.34      1493

    accuracy                           0.71     69057
   macro avg       0.59      0.55      0.56     69057
weighted avg       0.70      0.71      0.70     69057



In [53]:
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())

In [54]:
ada.fit(X_train_scaled, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier())

In [55]:
y_pred_ada = ada.predict(X_test_scaled)

In [56]:
print(classification_report(y_test, y_pred_ada))

              precision    recall  f1-score   support

           0       0.76      0.84      0.80     44262
           1       0.62      0.51      0.56     23302
           2       0.49      0.30      0.37      1493

    accuracy                           0.72     69057
   macro avg       0.62      0.55      0.58     69057
weighted avg       0.71      0.72      0.71     69057



In [57]:
from sklearn.neighbors import KNeighborsClassifier

In [58]:
knn = KNeighborsClassifier(n_neighbors = 5, metric = 'manhattan', weights = 'distance', leaf_size = 15)
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(leaf_size=15, metric='manhattan', weights='distance')

In [59]:
y_pred_knn = knn.predict(X_test_scaled)

In [60]:
print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

           0       0.76      0.81      0.79     44262
           1       0.59      0.53      0.56     23302
           2       0.47      0.29      0.36      1493

    accuracy                           0.71     69057
   macro avg       0.61      0.54      0.57     69057
weighted avg       0.70      0.71      0.70     69057

